In [1]:
from scrollguard.utils import get_config, get_mongo_client
from scrollguard import utils, etl
import numpy as np
from io import StringIO
import json
import requests
import ssl
import urllib3
import xmltodict
import pandas as pd
from Levenshtein import ratio

pd.set_option("display.max_column", None)

In [5]:
config = get_config().find_one({"source": "SOURCES"}, {"_id":0, "value":1})["value"]["UN_INDIVIDUAL"]
raw_data = etl.extract_xml(config)

In [6]:
transformer = etl.Transformer("UN_INDIVIDUAL", raw_data)